In [1]:
%pip install pymysql

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/0c/94/e4181a1f6286f545507528c78016e00065ea913276888db2262507693ce5/PyMySQL-1.1.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.0 kB ? eta -:--:--
   ------------------------------------ --- 41.0/45.0 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 560.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip show pymysql

Name: PyMySQL
Version: 1.1.1
Summary: Pure Python MySQL Driver
Home-page: 
Author: 
Author-email: Inada Naoki <songofacandy@gmail.com>, Yutaka Matsubara <yutaka.matsubara@gmail.com>
License: MIT License
Location: C:\Users\USER\anaconda3\Lib\site-packages
Requires: 
Required-by: 


In [1]:
sql = """
CREATE TABLE product (
    id INT UNSIGNED NOT NULL AUTO_INCREMENT,
    name VARCHAR(20) NOT NULL,
    model_num VARCHAR(10) NOT NULL,
    model_type VARCHAR(10) NOT NULL,
    PRIMARY KEY(id)
);
"""
sql

'\nCREATE TABLE product (\n    id INT UNSIGNED NOT NULL AUTO_INCREMENT,\n    name VARCHAR(20) NOT NULL,\n    model_num VARCHAR(10) NOT NULL,\n    model_type VARCHAR(10) NOT NULL,\n    PRIMARY KEY(id)\n);\n'

In [2]:
import pymysql

db = pymysql.connect(host='localhost', port=3307, db='python_db',\
                    user='python',passwd='python',charset='utf8')
print(type(db))
#cursor 객체 생성
cursor = db.cursor()
print(type(cursor))
#table을 생성하는 sql문을 실행
cursor.execute(sql)
#db에 적용시켜라
db.commit()

<class 'pymysql.connections.Connection'>
<class 'pymysql.cursors.Cursor'>


OperationalError: (1050, "Table 'product' already exists")

In [3]:
cursor.execute('show tables')

1

In [4]:
cursor.execute('drop table product')

0

In [5]:
db.close()

In [6]:
import pymysql

#db와 연결
db = pymysql.connect(host='localhost', port=3307, db='python_db',\
                    user='python',passwd='python',charset='utf8')

try:
    #cursor 생성하고 cursor가 open되어 있는 query문을 여러개 실행
    with db.cursor() as cursor:
        #table drop하는 query 실행
        #cursor.execute('drop table product')
        #product table 생성 query실행
        cursor.execute(sql)
        #db에 실제로 적용한다
        db.commit()
        
        for num in range(10,20): #10~19
            name = 'S20'+str(num)
            ins_sql = \
            'insert into product (name,model_num,model_type) values (%s, %s, %s)'
            cursor.execute(ins_sql,(name,'7700','Phone'))
            print(ins_sql)
#             ins_sql = "insert into product (name,model_num,model_type) \
#             values('"+name+"','7700','Phone')"
#             cursor.execute(ins_sql)
            
        db.commit()
        print(cursor.lastrowid)
except Exception as exp:
    print(exp)
    db.rollback()
finally:
    db.close()

insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
insert into product (name,model_num,model_type) values (%s, %s, %s)
10


In [7]:
import pymysql

db = pymysql.connect(host='localhost', port=3307, db='python_db',\
                    user='python',passwd='python',charset='utf8')

try:
    #select, update
    with db.cursor() as cursor:
        cursor.execute('select * from product where id=3')
        result = cursor.fetchone()
        print(type(result),result, result[1])
        
        upd_sql = \
        "update product set model_type='%s' \
        where name between 'S2010' and 'S2015'" % '핸드폰'
        cursor.execute(upd_sql)
        db.commit()
        #갱신된 row 갯수
        print(cursor.rowcount)
        
        cursor.execute('select * from product')
        result_list = cursor.fetchall()
        print(type(result_list))
        for row in result_list:
            print(row[0],row[1],row[2],row[3])
            
        # model_type별로 group by 하는 쿼리 실행 
        cursor.execute('select model_type,count(*) from product group by model_type')
        for row in cursor.fetchall():
            print(row)
finally:
    db.close()

<class 'tuple'> (3, 'S2012', '7700', 'Phone') S2012
6
<class 'tuple'>
1 S2010 7700 핸드폰
2 S2011 7700 핸드폰
3 S2012 7700 핸드폰
4 S2013 7700 핸드폰
5 S2014 7700 핸드폰
6 S2015 7700 핸드폰
7 S2016 7700 Phone
8 S2017 7700 Phone
9 S2018 7700 Phone
10 S2019 7700 Phone
('Phone', 4)
('핸드폰', 6)


In [30]:
# delete 하고 select all
#name 컬럼의 값이 's2014' 와 's2015' 인 행을 삭제하세요 sql의 in 구문을 사용하세요

con = pymysql.connect(host='localhost', port=3306, db='python_db',\
                    user='python',passwd='python',charset='utf8')

#print(type(con), con)

try:
    with con.cursor() as cursor:
        sql = "delete from product where name in (%s,%s)"
        cursor.execute(sql,('S2012','S2013'))
        con.commit()
        # 삭제된 건수 출력
        print(cursor.rowcount)
                
        sql = "select * from product order by id"
        cursor.execute(sql)
        for row in cursor.fetchall():
            print(row[0],row[1],row[2],row[3])
            
except Exception as ex:
    con.rollback()
    print(ex)
finally:
    con.close()

0
7 S2016 7700 Phone
8 S2017 7700 Phone
9 S2018 7700 Phone
10 S2019 7700 Phone
